#Active Learning Selection of training file

Active learning originaly aims to maximize the information gained from labeling at a limited annotation budget. In this applictaion, as we noticed the classes are highley imbalanced, and the presence of images and large objects all problems thata could hender the model's ability to generalize at deployment.

In this Notebook we present the Weighing by Random Performance Sum of Averages Active learning approach proposed. 



1.   It first evaluate the model from the previous iteration on a random subset of the training data that hasn't been used for trianing yet.
2.   It then calculated the uncertinity of the inference made on the training data that hasn't been used for trianing yet, by summing the average uncertinities per class for eaach image
3.  After that it multiplies the weights with uncertinities and produce the final uncertinity score per image.
4. Finally, the images with the highest uncertinities are included in the new training cycle.
5. I model is build using the selected images using the notebook in [here](https://github.com/asmayamani/GreenHawk/blob/main/GreenHawk_training.ipynb)


* In some iterations we included some of the images in which were not detected at all by the model seetting their uncertinities to +inf
* We also included all images containing the 5 undererepresented classes explicitly.




#Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)


Mounted at /content/drive


In [ ]:
import os, glob, random, shutil
from pathlib import Path    

In [ ]:
!git clone https://github.com/asmayamani/ultralytics-AL.git
%cd ultralytics-AL
!pip install -e '.[dev]'


Cloning into 'ultralytics-AL'...
remote: Enumerating objects: 2766, done.
remote: Total 2766 (delta 0), reused 0 (delta 0), pack-reused 2766
Receiving objects: 100% (2766/2766), 1.42 MiB | 12.00 MiB/s, done.
Resolving deltas: 100% (1889/1889), done.
/content/ultralytics-AL
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/ultralytics-AL
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.3 

#Declerations

Important: Increase the itereation after completing each iteration

In [ ]:
#Enter the number iteration
iteration = 18

In [ ]:

main_dir = '/content/drive/MyDrive/GreenHawak/Dataset/'
complete_data_folder = main_dir+'TrainF/'
main_exp_dir = 'WRP/'
iteration_folder = 'Iteration_'+str(iteration)+'/'
complete_iteration_folder = main_dir+main_exp_dir+iteration_folder
os.makedirs(complete_iteration_folder) if not os.path.exists(complete_iteration_folder) else None
object_def = {'graffiti':0,'faded_signage':1,'potholes':2,'garbage':3,
                   'construction_road':4,'broken_signage':5,
                   'bad_streetlight':6,'bad_billboard':7,'sand_on_road':8,'clutter_sidewalk':9,'unkept_facade':10}
approach = 'SumAvg' 
weighting_approach = 'WRP'
number_of_random_images_from_training = 6000
output_model = approach+str(iteration-1)+str(number_of_random_images_from_training)
previous_model_zip = 'Iteration_'+str(iteration-1)+'_W.zip'
budgetPerIteration = 300
expName = 'iteration'+approach+str(iteration)+'_'+str(number_of_random_images_from_training)
wexpName = 'w_iteration'+approach+str(iteration)+'_'+str(number_of_random_images_from_training)
previousExpfName = 'iteration'+str(iteration-1)+approach+weighting_approach+'_'+str(number_of_random_images_from_training)+'/'
previous_iteration_folder = 'Iteration_'+str(iteration-1)+'/'


#Definitions

The methods for active learning and some utility functions are defined here 

In [ ]:
def sumAvgUncertinitiesForFile(fDir, classesWeight):
    classes = {}
    for i in range(len(object_def)):
      classes[str(i)] = {'objectsC' : 0, 'uncer' : 0}
    try:
      with open(fDir, 'r') as file2:  # read annotation.txt
          for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
              classes[str(row[0])]['objectsC'] += 1
              classes[str(row[0])]['uncer'] += ((1-float(row[5])))
      file2.close()
      total = 0
      wAvg = 0
      for i in range(len(object_def)):
        # print(classes[str(i)])
        if classes[str(i)]['objectsC'] != 0:
          wAvg += ((classes[str(i)]['uncer']/classes[str(i)]['objectsC']) * classesWeight[str(i)])
          total += classes[str(i)]['objectsC'] 

      return(wAvg)
    except (IOError, OSError) as e:
        print(e)
        return(0)


In [ ]:
def getUncertinitiesForDir(dir, trainDir = [], valAccuracyDirF = '', method = 'avg', weighting = 1 , budget = 100):
  classesWeight = getWeightsForClassses(weighting, trainDir = trainDir, valAccuracyDirF = valAccuracyDirF)
  listofFilesNamesUncertinity = {}
  for f in glob.glob(dir+'/labels/*.txt'):
    namef = Path(f).name[:-4]
    # listofFilesNames.append(namef)
    fileFullDir = dir+'/labels/'+namef+'.txt'
    # if method == 'sum':
    #   listofFilesNamesUncertinity[namef] = sumUncertinitiesForFile(fileFullDir,classesWeight)
    # if method == 'avg':
    #   listofFilesNamesUncertinity[namef] = avgUncertinitiesForFile(fileFullDir,classesWeight)
    if method == 'sumAvg':
      listofFilesNamesUncertinity[namef] = sumAvgUncertinitiesForFile(fileFullDir,classesWeight)
  print(listofFilesNamesUncertinity)
  return sorted(listofFilesNamesUncertinity, key=listofFilesNamesUncertinity.get, reverse=True)[:budget]


In [ ]:
def getWeightsForClassses(weighting, trainDir = [], valAccuracyDirF = '', ):
  classesWeight = {}
  if weighting == 1:
    for i in range(len(object_def)):
      classesWeight[str(i)] = 1
  # elif weighting == 2:
  #   classesWeight = getWeightForTrainingLabelsPerClass(trainDir)
  elif weighting == 3:
    classesWeight = applyMinMaxScalerToW(getWeightForValAccuracy(valAccuracyDirF))
  print(classesWeight)
  return classesWeight


In [ ]:
def newTraining(namefolder,setOfFileNames,oldTraning):
  added = 0
  path_label_2 = complete_iteration_folder+namefolder+'/labels/'
  path_images_2 = complete_iteration_folder+namefolder+'/images/'
  os.makedirs(path_label_2) if not os.path.exists(path_label_2) else None
  os.makedirs(path_images_2) if not os.path.exists(path_images_2) else None
  count = 0
  #why 100?
  while (added < budgetPerIteration*2) and (count < len(setOfFileNames)):
    namef = setOfFileNames[count] #Path(setOfFileNames[count]).name[:-4]
    
    l_src_path = complete_data_folder+'labels/'+ namef+'.txt'
    i_src_path = complete_data_folder+'images/'+ namef+'.jpg'

    src_path = complete_data_folder+'labels/'+ namef+'.txt'
    if os.path.exists(l_src_path) and os.path.exists(i_src_path):
      dst_path = path_label_2 + namef+'.txt'
      shutil.copy(src_path, dst_path)
      src_path = complete_data_folder+'images/'+ namef+'.jpg'
      dst_path = path_images_2 + namef+'.jpg'
      shutil.copy(src_path, dst_path)
      added += 1
    count += 1

  for pth in oldTraning:
    # print(pth)
    namef = Path(pth).name[:-4]
    src_path = main_dir+main_exp_dir+previous_iteration_folder+previousExpfName+'labels/'+ namef+'.txt'
    dst_path = path_label_2 + namef+'.txt'
    shutil.copy(src_path, dst_path)
    src_path = main_dir+main_exp_dir+previous_iteration_folder+previousExpfName+'images/'+ namef+'.jpg'
    dst_path = path_images_2 + namef+'.jpg'
    shutil.copy(src_path, dst_path)


In [ ]:
def getWeightForValAccuracy(fDir, cnum = object_def):
    numberOfClasses = len(object_def)
    accuracyClasses = {}
    for i in range(numberOfClasses):
      accuracyClasses[str(i)] = 0
    try:
      with open(fDir, 'r') as file2:  # read annotation.txt
          first = True
          for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
            if first:
              first = False 
              continue
            row = list(filter(lambda a: a != '', row))
            accuracyClasses[str(cnum[row[0]])] = float(row[5])
            
            
      file2.close()
      return(accuracyClasses)
    except (IOError, OSError) as e:
        print(e)


In [ ]:
def applyMinMaxScalerToW (wValCDic):
  minV = min(wValCDic[item] for item in wValCDic)
  maxV = max(wValCDic[item] for item in wValCDic)
  for i in range(len(object_def)):
    wValCDic[str(i)] = 1- ((wValCDic[str(i)] - minV)/(maxV - minV))
  return wValCDic

In [ ]:
def nominateImagesToObtainUncer(wasTrainedOnDir, experimentName, forWeights = False, fromRandSize = 500, seed = 42, trainingDir = complete_data_folder+'images/', weightedOnDir = '' ):
  #remove images that were trained on
  trainedOn = glob.glob(wasTrainedOnDir+'/images/*.jpg')
  fullTrainSet = glob.glob(trainingDir+'*.jpg')
  orpNames = [Path(f).name[:-4] for f in trainedOn]
  if weightedOnDir != '':
    weightedOn = glob.glob(weightedOnDir+'/images/*.jpg')
    orpNames = orpNames + [Path(f).name[:-4] for f in weightedOn]

  mixedRand = []
  for ele in fullTrainSet:
    for ele2 in orpNames:
      if ele2 in ele:
        mixedRand.append(ele)
        continue
  randomPathes2 = [ele for ele in fullTrainSet if ele not in mixedRand]

  reducedRandomPathes2 = []
  for rp2 in randomPathes2:
    if os.path.exists(complete_data_folder+'labels/'+ Path(rp2).name[:-4] + '.txt'):
        reducedRandomPathes2.append(rp2)
  toGetUncertinityPath = reducedRandomPathes2
  if fromRandSize < number_of_random_images_from_training:
    toGetUncertinityPath = random.sample(reducedRandomPathes2, k=fromRandSize)

  newFolderName = experimentName+'_Nominees'
  path_label = complete_iteration_folder+newFolderName+'/labels/'
  path_images = complete_iteration_folder+newFolderName+'/images/'
  os.makedirs(path_label) if not os.path.exists(path_label) else None
  os.makedirs(path_images) if not os.path.exists(path_images) else None

  for pth in toGetUncertinityPath:
    namef = Path(pth).name[:-4]
    if forWeights:
      src_path = complete_data_folder+'labels/'+ namef+'.txt'
      dst_path = path_label + namef+'.txt'
      shutil.copy(src_path, dst_path)
    src_path = complete_data_folder+'/images/'+ namef+'.jpg'
    dst_path = path_images + namef+'.jpg'
    shutil.copy(src_path, dst_path)
  return path_images


#Test on Random subset
In this part we select some random validation subset to evaluate the peformance on and subsequently weigh the uncertinitiies of the different classes

In [ ]:
path_images_nominees_test = nominateImagesToObtainUncer(main_dir+main_exp_dir+previous_iteration_folder+previousExpfName,wexpName,fromRandSize=int(budgetPerIteration*0.1),forWeights=True)


In [ ]:
!unzip {main_dir+main_exp_dir+previous_iteration_folder}{previous_model_zip} -d /content/ultralytics-AL/ultralytics/{previous_iteration_folder}

Archive:  /content/drive/MyDrive/Smartathon/WRP/Iteration_17/Iteration_17_W.zip
  inflating: /content/ultralytics-AL/ultralytics/Iteration_17/content/runs/detect/train/weights/best.pt  


In [ ]:
#This somtims has to be updated manually based on the unzipped file above
weights_path = previous_iteration_folder+'/content/runs/detect/train/weights/best.pt'

In [ ]:
import yaml
config = {
        #  'path': '../datasets/Smartathon',
        'train': '/content/drive/MyDrive/GreenHawk/Dataset/WRP/Iteration_0/images',
         'val': complete_iteration_folder+wexpName+'_Nominees/images',
         'nc': 11,
         'names': ['graffiti','faded_signage','potholes','garbage',
                   'construction_road','broken_signage',
                   'bad_streetlight','bad_billboard','sand_on_road','clutter_sidewalk','unkept_facade']}

with open("/content/ultralytics-AL/ultralytics/data.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)


In [ ]:
%cd ultralytics
!yolo task=detect mode=val data=data.yaml model={weights_path}  

/content/ultralytics-AL/ultralytics
Ultralytics YOLOv8.0.9 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Fusing... 
Model summary: 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs
100% 755k/755k [00:00<00:00, 16.1MB/s]
val: Scanning /content/drive/MyDrive/Smartathon/WRP/Iteration_18/w_iterationSumAvg18_6000_Nominees/labels... 30 images, 0 backgrounds, 0 corrupt: 100% 30/30 [00:00<00:00, 136.51it/s]
val: New cache created: /content/drive/MyDrive/Smartathon/WRP/Iteration_18/w_iterationSumAvg18_6000_Nominees/labels.cache
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:12<00:00,  6.36s/it]
                   all         30        119      0.569      0.666      0.661      0.306
              graffiti         30          6      0.521      0.667      0.628      0.274
              potholes       

#Calculating the uncertinity

In [ ]:
path_images_nominees = nominateImagesToObtainUncer(main_dir+main_exp_dir+previous_iteration_folder+previousExpfName,expName, fromRandSize = number_of_random_images_from_training, weightedOnDir=complete_iteration_folder+wexpName+'_Nominees')


In [ ]:
!yolo task=detect mode=predict source={main_dir+main_exp_dir+iteration_folder+expName+'_Nominees/images/'} model={weights_path} imgsz=256 plots=False  save_txt=True save_conf=True save_crop=False

Ultralytics YOLOv8.0.9 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Fusing... 
Model summary: 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs
image 1/1940 /content/drive/MyDrive/Smartathon/WRP/Iteration_18/iterationSumAvg18_6000_Nominees/images/000b0fd767c5ab2656b7ed8d43674777.jpg: 160x256 1 garbage, 90.6ms
image 2/1940 /content/drive/MyDrive/Smartathon/WRP/Iteration_18/iterationSumAvg18_6000_Nominees/images/0019a9ab6fd5ef3df915241bc323afb5.jpg: 160x256 2 construction_roads, 74.4ms
image 3/1940 /content/drive/MyDrive/Smartathon/WRP/Iteration_18/iterationSumAvg18_6000_Nominees/images/004a08c20af8b043a5a1cdfc6a01b48a.jpg: 160x256 6 garbages, 71.7ms
image 4/1940 /content/drive/MyDrive/Smartathon/WRP/Iteration_18/iterationSumAvg18_6000_Nominees/images/00b5bf7dce31145b95ef3adb89a99b03.jpg: 160x256 3 construction_roads, 73.2ms
image 5/1940 /content/drive/MyDrive/Smartathon/WRP/Iteration_18/iterationSumAvg18_6000_Nominees/images/00bb84f0f10393189e229e52f4b2f652.jpg: 160x256 2 graffitis, 7

#Weighing the uncertinities

In [ ]:
expfName = 'iteration'+str(iteration)+approach+weighting_approach+'_'+str(number_of_random_images_from_training)
predFile = '../runs/detect/predict'
wafile = '../runs/detect/val/results.txt'


In [ ]:
previousFileList = glob.glob(main_dir+main_exp_dir+previous_iteration_folder+previousExpfName+'labels/*.txt')
trainOnSumAvg = getUncertinitiesForDir(predFile,valAccuracyDirF= wafile, method='sumAvg',budget = int(budgetPerIteration),weighting = 3,)

{'0': 0.3688442211055276, '1': 1.0, '2': 0.0, '3': 0.5386934673366834, '4': 0.3296482412060301, '5': 1.0, '6': 1.0, '7': 1.0, '8': 0.5678391959798995, '9': 0.20904522613065324, '10': 1.0}
{'2c57ec74a93f0a9943884138cdc370ab': 0.42398716247906193, 'a33a221e5394da9294b9b3a6c815608b': 0.8772927226130653, '32ba8f6cf9f1ec9c73e33c8ba4a1b207': 0.45374223115577883, '4bdbaa786ab013638ddeba2d2db45960': 0.15151973065326635, '7bbcc988ad2880d2e7d0485389b57b37': 0.1881866211055276, '52ed106250d102409774a6371c0cb162': 0.27940198592964827, 'c4218b61f2d1280d5d359cdd5c234a3f': 0.1337498291457286, 'c4cfcde2821ffc935f5d5cbf62cd3ead': 0.40330363819095477, '9983c15208506f5d8f4dea4d53f5d367': 0.0, 'bc2e9e57a48ebccec901fb8a3822a1eb': 0.20251829815745392, 'c19de2d70bb58cf7ec8fae9539b26a1d': 0.5048214110552764, '50a9ca0f1c64c1f9b468c74dc703f9d0': 0.12003676515912898, '5f2166bc7688d4dbb334418912c17b2e': 0.48633096147403687, '59b3fc0be26c4dfeb7e2c961a5669d0e': 0.33278042900215354, '651ea3beb46b1fb0d56f9e2c7d0163dd

#Adding some of the images with undetected lables (As they have inf uncertinity)

In [ ]:
detected = [Path(pth).name[:-4] for pth in glob.glob(predFile+'/labels/*.txt')]
allNotTrained = [Path(pth).name[:-4] for pth in (glob.glob(predFile+'/*.jpg'))]
undetected = list(set(allNotTrained) - set(detected))

#Construct the new train set for the next iteration

In [ ]:
newTraining(expfName,trainOnSumAvg+undetected[:200],previousFileList)

In [ ]:
path_label = complete_iteration_folder+expfName+'/labels/'
path_images = complete_iteration_folder+expfName+'/images/'

wimg = glob.glob(complete_iteration_folder+wexpName+'_Nominees/images/*')
wtxt = glob.glob(complete_iteration_folder+wexpName+'_Nominees/labels/*')
for pth in wtxt:
  namef = Path(pth).name[:-4]
  src_path = complete_iteration_folder+wexpName+'_Nominees/labels/'+ namef+'.txt'
  dst_path = path_label + namef+'.txt'
  shutil.copy(src_path, dst_path)
for pth in wimg:
  namef = Path(pth).name[:-4]
  src_path = complete_iteration_folder+wexpName+'_Nominees/images/'+ namef+'.jpg'
  dst_path = path_images + namef+'.jpg'
  shutil.copy(src_path, dst_path)

In [ ]:
orp2 = glob.glob(complete_iteration_folder+expfName+'/images/*.jpg')
print(len(orp2))

5586


In [ ]:
from google.colab import runtime
runtime.unassign()

#Getting the images underrepresented classes (done once) to be augmented on RoboFlow

In [ ]:
def containsUnderRepresentedClasses(namedir):
  urc = [10,1,5,2,6,8]
  try:
    with open(namedir, 'r') as file2:  # read annotation.txt
      for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
          if int(row[0]) in urc:
            print(row[0])
            return True
            file2.close()
      return False
  except (IOError, OSError) as e:
      return False


In [ ]:
containsUnderRepresentedClassesList = []
nominatedName = [Path(namedir).name[:-4] for namedir in glob.glob(main_dir+main_exp_dir+iteration_folder+expName+'_Nominees/images/*.jpg')]
nominatedDir = [complete_data_folder+'labels/'+ namef+'.txt' for namef in nominatedName]
nnomD = glob.glob('/content/drive/MyDrive/GreenHawk/Dataset/TrainF/'+'/labels/*.txt')
c = 0
f = 0
for namedir in nnomD:

  f = f + 1
  if containsUnderRepresentedClasses(namedir):
    c = c + 1
    containsUnderRepresentedClassesList.append(Path(namedir).name[:-4])
  

2
10
2
10
2
1
8
8
5
2
8
2
2
2
8
8
2
8
8
2
8
2
5
2
10
2
2
2
2
2
2
2
2
8
2
2
5
2
8
2
2
2
2
8
2
8
8
2
8
8
2
2
10
8
8
2
2
2
2
1
2
2
8
2
10
2
2
2
8
8
1
2
2
2
5
2
2
8
2
8
8
2
8
2
2
5
8
2
2
2
2
2
8
2
2
2
2
2
2
8
2
8
2
8
2
8
2
2
2
8
2
2
8
2
2
2
2
2
2
8
2
2
1
2
8
10
2
2
2
2
8
2
2
2
8


In [29]:
len(containsUnderRepresentedClassesList)

168

In [31]:

os.makedirs(complete_iteration_folder+'rare/labels/') if not os.path.exists(complete_iteration_folder+'rare/labels/') else None
os.makedirs(complete_iteration_folder+'rare/images/') if not os.path.exists(complete_iteration_folder+'rare/images/') else None

for rareName in containsUnderRepresentedClassesList:

  src_path = complete_data_folder+'labels/'+ rareName+'.txt'
  print(src_path)
  dst_path = complete_iteration_folder+'rare/labels/' + rareName+'.txt'
  shutil.copy(src_path, dst_path)

  src_path = complete_data_folder+'images/'+ rareName+'.jpg'
  dst_path = complete_iteration_folder+'rare/images/' + rareName+'.jpg'
  shutil.copy(src_path, dst_path)


/content/drive/MyDrive/Smartathon/ValF/labels/f05cd6411a3509a5ddc9d9a52536df01.txt
/content/drive/MyDrive/Smartathon/ValF/labels/e3a2998c769f43f2767cce3c50e0e193.txt
/content/drive/MyDrive/Smartathon/ValF/labels/4255fa84582ba207069d415202d520ae.txt
/content/drive/MyDrive/Smartathon/ValF/labels/34b8b29c7cb140b84192b342437e5355.txt
/content/drive/MyDrive/Smartathon/ValF/labels/e988d56a045315b828343821940259c7.txt
/content/drive/MyDrive/Smartathon/ValF/labels/2bb02e9237647d20bf2188ba7b798f0a.txt
/content/drive/MyDrive/Smartathon/ValF/labels/c4b21e494a068b613c12a0a4e149a106.txt
/content/drive/MyDrive/Smartathon/ValF/labels/00f54ee82124ff35d346d15145475ab4.txt
/content/drive/MyDrive/Smartathon/ValF/labels/ac0e3cb82cae8d807c0b3a833832e2fd.txt
/content/drive/MyDrive/Smartathon/ValF/labels/2053ccdefe61c3bf36923aca3a042c26.txt
/content/drive/MyDrive/Smartathon/ValF/labels/bda5447b4ec7bcafa823e22f7187cc8d.txt
/content/drive/MyDrive/Smartathon/ValF/labels/f44bea67dd5c663b891f1864d357f244.txt
/con